In [4]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate, ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template(
    "What is the distance between {country_a} and {country_b}"
)
prompt = template.format(country_a="Mexico", country_b="Thailand")

chat.predict(prompt)


'The distance between Mexico and Thailand is approximately 16,000 kilometers (9,942 miles) when measured in a straight line.'

In [6]:

template = ChatPromptTemplate.from_messages([
    ("system", "You are a geography expert. And you only reply in {language}."),
    ("ai", "Ciao, mi chiamo {name}!"),
    ("human", "What is the distance between {country_a} and {country_b}. Also, what is your name?")
])

prompt = template.format_messages(
    language="Greek",
    name="Socrates",
    country_a="Mexico",
    country_b="Thailand"
)

chat.predict_messages(prompt)

AIMessage(content='Γεια σας! Η απόσταση μεταξύ του Μεξικού και της Ταϊλάνδης είναι περίπου 16.000 χιλιόμετρα. Το όνομά μου είναι Σωκράτης. Πώς μπορώ να βοηθήσω;')

In [8]:
from langchain.schema import BaseOutputParser

class CommaOutputParser(BaseOutputParser):

    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))

p = CommaOutputParser()
p.parse("Hello,how,are, you?")

['Hello', 'how', 'are', 'you?']

In [ ]:
template = ChatPromptTemplate.from_messages([
    ("system", "You are a list generating machine. Everything you are asked will be answered with a comma separated list of max {max_items} in lowercase. Do NOT reply with anything else."),
    ("human", "{question}"),
])

# prompt = template.format_messages(
#     max_items=10,
#     question="What are the colors?"
# )

# result = chat.predict_messages(prompt)

# p = CommaOutputParser()

# p.parse(result.content)

In [14]:
chain = template | chat | CommaOutputParser()

chain.invoke({"max_items": 5, "question": "What are the pokemons?"})

['pikachu', 'charmander', 'bulbasaur', 'squirtle', 'jigglypuff']

In [20]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international chef. You create easy to follow recipies for any type of cuisine with easy to find ingredients."),
    ("human", "I want to cook {cuisine} food."),
])

chef_chain = chef_prompt | chat

veg_chef_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a vegetarian chef specialized on making traditional recipes vegetarian. You find alternative ingredients and explain their preparation. You don't radically modify the recipe. If there is no alternative for a food just say you don't know how to replace it."),
    ("human", "{recipe}")
])

veg_chain = veg_chef_prompt | chat

final_chain = {"recipe": chef_chain} | veg_chain

final_chain.invoke({
    "cuisine": "indian"
})

Great choice! Indian cuisine is known for its bold flavors and aromatic spices. Let's start with a classic and popular dish - Chicken Tikka Masala. Here's a simple recipe for you to try at home:

Ingredients:
- 1 lb boneless, skinless chicken breasts, cut into bite-sized pieces
- 1 cup plain yogurt
- 2 tablespoons lemon juice
- 2 tablespoons vegetable oil
- 1 onion, finely chopped
- 3 cloves garlic, minced
- 1-inch piece of ginger, grated
- 1 can (14 oz) diced tomatoes
- 2 tablespoons tomato paste
- 1 tablespoon garam masala
- 1 teaspoon ground cumin
- 1 teaspoon ground coriander
- 1 teaspoon paprika
- 1/2 teaspoon turmeric
- 1/2 teaspoon cayenne pepper (adjust to taste)
- Salt and pepper to taste
- Fresh cilantro, chopped for garnish
- Cooked rice or naan bread for serving

Instructions:
1. In a bowl, combine the yogurt, lemon juice, 1 tablespoon of vegetable oil, half of the minced garlic, half of the grated ginger, and a pinch of salt. Add the chicken pieces and mix well to coat. Co

AIMessageChunk(content="As a vegetarian chef specialized in making traditional recipes vegetarian, I can offer you a delicious alternative to Chicken Tikka Masala using paneer instead of chicken. Paneer is a fresh cheese commonly used in Indian cuisine and is a great substitute for meat in many dishes. Here's how you can adapt the recipe:\n\nIngredients:\n- 1 lb paneer, cut into bite-sized pieces\n- 1 cup plain yogurt\n- 2 tablespoons lemon juice\n- 2 tablespoons vegetable oil\n- 1 onion, finely chopped\n- 3 cloves garlic, minced\n- 1-inch piece of ginger, grated\n- 1 can (14 oz) diced tomatoes\n- 2 tablespoons tomato paste\n- 1 tablespoon garam masala\n- 1 teaspoon ground cumin\n- 1 teaspoon ground coriander\n- 1 teaspoon paprika\n- 1/2 teaspoon turmeric\n- 1/2 teaspoon cayenne pepper (adjust to taste)\n- Salt and pepper to taste\n- Fresh cilantro, chopped for garnish\n- Cooked rice or naan bread for serving\n\nInstructions:\n1. Follow the same marinating process as the original recip

In [21]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler

chat = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

poet_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world-class international Poet. You create poetry about programming languages. you use only one programming langauage to one poetry"),
    ("human", "I want to make {language} poetry."),
])

poet_chain = poet_prompt | chat

poetry_commentator_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a poetry commentator specialized on explaining programming language's poetry."),
    ("human", "{poetry}")
])

poetry_commentator_chain = poetry_commentator_prompt | chat

final_chain = {"poetry": poet_chain} | poetry_commentator_chain

final_chain.invoke({
    "language": "python"
})

In the land of code, where serpents slither,
Python reigns, a language that's neither
Too complex nor too simple, it strikes a balance,
With syntax clean and elegant, it's a true romance.

Indentation matters, in Python's domain,
No curly braces, just whitespace to maintain,
From loops to functions, it's all so clear,
Python's beauty shines, without any fear.

Libraries galore, for every need,
NumPy, Pandas, and more indeed,
Machine learning, data science, and AI,
Python's versatility soars high in the sky.

So let us embrace this language so grand,
With Python in hand, we'll conquer the land,
For in the world of programming, it's plain to see,
Python's power and grace will forever be.This poem beautifully captures the essence of Python, a programming language known for its simplicity, elegance, and versatility. The poet uses vivid imagery and metaphors to paint a picture of Python as a powerful and graceful language that reigns supreme in the land of code.

The mention of serpents sli

AIMessageChunk(content="This poem beautifully captures the essence of Python, a programming language known for its simplicity, elegance, and versatility. The poet uses vivid imagery and metaphors to paint a picture of Python as a powerful and graceful language that reigns supreme in the land of code.\n\nThe mention of serpents slithering in the land of code sets the tone for Python's dominance, likening it to a majestic ruler. The poet praises Python for striking a perfect balance between complexity and simplicity, making it accessible yet powerful.\n\nThe emphasis on indentation in Python's domain highlights the unique feature of the language, where whitespace is used to define code blocks instead of curly braces. This attention to detail and cleanliness in syntax is described as a true romance, showcasing the poet's admiration for Python's beauty.\n\nThe poem also celebrates Python's rich ecosystem of libraries, such as NumPy and Pandas, that cater to various needs like machine learn